In [7]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import os
import sys
import time

import tensorflow.python.platform

import numpy as np
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf

from tensorflow.python.platform import gfile


def _read_words(filename):
  with gfile.GFile(filename, "r") as f:
    return f.read().replace("\n", "<eos>").split()

data = _read_words('data/chars.txt')
counter = collections.Counter(data)
count_pairs = sorted(counter.items(), key=lambda x: -x[1])
words, _ = list(zip(*count_pairs))
word_to_id = dict(zip(words, range(len(words))))
print(word_to_id)

{'gold': 43, 'show': 57, 'consolidated': 45, 'results': 46, 'years': 9, 'bring': 47, 'university<eos>the': 48, 'rubens': 49, 'including': 50, 'filters': 10, 'its': 51, 'before': 52, 'centrust': 147, 'enters': 53, 'group': 54, 'surviving': 55, 'banknote': 56, 'workers': 15, 'than': 16, 'symptoms': 58, 'to': 6, 'now<eos>neither': 59, 'board': 60, 'caused': 61, 'has': 62, 'decades': 63, 'risk': 65, 'asbestos': 7, 'rake': 66, 'ipo': 135, 'early': 68, 'five': 69, 'closely': 70, 'swapo': 104, 'schools': 71, 'using': 72, 'corp.': 73, 'nor': 74, 'aer': 76, 'fields': 77, 'fromstein': 78, 'whether': 79, 'paper': 80, 'stopped': 81, 'reported<eos>the': 82, 'james': 83, 'university': 84, 'old': 18, 'billion': 85, 'findings': 86, 'exposed': 87, 'national': 88, 'calloway': 89, 'likely': 90, 'are': 91, 'N<eos>mr.': 92, 'our': 93, 'even': 94, 'will': 163, 'said': 19, 'appear': 97, 'for': 98, 'brief': 99, 'gitano': 100, 'research': 101, 'kent': 11, 'dutch': 102, 'medicine': 103, 'new': 17, 'cigarettes<e

In [10]:
data = _read_words('data/chars.txt')
ids = [word_to_id[word] for word in data]
print(ids[0])

76


In [12]:
raw_data = np.array(ids, dtype=np.int32)
data_len = len(raw_data)
print(data_len)
batch_len = data_len // 30
print(batch_len)

348
11


In [16]:
batch_size = 30
iter_data = np.zeros([batch_size, batch_len], dtype=np.int32)
print(raw_data)
print(raw_data[batch_len * 0:batch_len * (0 + 1)])

[ 76  56 136  89 147 192  78 100 158 171 135 197 179 168  44 116  66 154
  49 189 141 199 104 107   0   4   9  18 163 160   1  60 177   3  34  39
 159  92   0  13  35   2   0 139   1 102 140 181   0   4   9  18  12 157
  35   2  45  43  77 117  30 132   3  34  39   2  29 125 133 149 152   2
   7  42  36   6 186  11  40  10  62  61   3 175 146   2  20 172 131   3
  54   2  15  87   6  23  21  16   4   9  25  14  82   7 109   0  13 167
   0  42  23  53   1   0   8  94  99 169   6  23 188  58  28  57 162  63
 202  14 148 203   1 138   2  17 185   0  73  28 130  11  41  81  72   0
   5  51   0  40  10   5  67 196  86  22 115  21  16   3 173  25   1 204
  46  97   5 166  37  17 137 113   2 103   3 122  90   6  47  17 114   6
   1  75   0   0  19  29  13 174  18 108 164 176 120   9  25  52 143 161
   2   7 205  32 150 183  13  33   7   5  93 128  59   0  74   1  14  38
 124   1  15  22 198   2  32 101  26 129   2   1  11 105  24  33 134 191
  26  79 190  91 178  65  19  83 193   0   2  31  3

In [17]:
for i in range(batch_size):
    iter_data[i] = raw_data[batch_len * i:batch_len * (i + 1)]
num_steps = 3
epoch_size = (batch_len - 1) // num_steps
print(epoch_size)

3


In [18]:
print(iter_data[:, 0*num_steps:(0+1)*num_steps])

[[ 76  56 136]
 [197 179 168]
 [104 107   0]
 [  3  34  39]
 [  1 102 140]
 [  2  45  43]
 [ 29 125 133]
 [ 11  40  10]
 [131   3  54]
 [  9  25  14]
 [ 23  53   1]
 [ 58  28  57]
 [  2  17 185]
 [  0   5  51]
 [115  21  16]
 [166  37  17]
 [ 47  17 114]
 [174  18 108]
 [  2   7 205]
 [128  59   0]
 [198   2  32]
 [ 33 134 191]
 [193   0   2]
 [ 64   2  14]
 [ 71   2 127]
 [ 30  36   5]
 [ 10   5   1]
 [  2   0   5]
 [  8   1  10]
 [156  70   8]]


In [23]:
for i in range(epoch_size):
    x = iter_data[:, i*num_steps:(i+1)*num_steps]
    y = iter_data[:, i*num_steps+1:(i+1)*num_steps+1]